In [1]:
!pip install dfply
import pandas as pd
import numpy as np
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statistics 
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import pearsonr
import plotly.graph_objects as go
import warnings
from scipy.stats import shapiro
from datetime import date, timedelta
from IPython.display import Image
from dateutil.relativedelta import *
import datetime as dt

     |████████████████████████████████| 614kB 2.8MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")

In [0]:
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Eda/markval_prev_eda.pkl")

In [0]:
import sys
sys.path.append("/content/drive/My Drive/Capstone/Helpers/EDA/")

In [0]:
from mv_eda_helpers import *
#Importing helpers

In [0]:
# markval = ready_for_eda(markval, players)
# The preperation functions were run once and the output was saved

In [0]:
markval.dropna().to_pickle("/content/drive/My Drive/Capstone/Data/Eda/markval_eda_data.pkl")

In [8]:
markval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72773 entries, 1408 to 87841
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   tm_id                72773 non-null  int64         
 1   club                 72773 non-null  object        
 2   league               72773 non-null  object        
 3   season               72773 non-null  category      
 4   mv                   72773 non-null  float64       
 5   name                 72773 non-null  object        
 6   dob                  72453 non-null  datetime64[ns]
 7   nationality          72773 non-null  object        
 8   continent            72773 non-null  category      
 9   field_position       72773 non-null  category      
 10  main_field_position  72773 non-null  category      
 11  age                  72453 non-null  float64       
 12  year                 72773 non-null  int64         
 13  league_class         72773 n

In [9]:
mv = markval.mv
markval_out = markval[~markval.mv.between(mv.quantile(.15), mv.quantile(.85))]
markval_norm = markval[markval.mv.between(mv.quantile(.15), mv.quantile(.85))]
markval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72773 entries, 1408 to 87841
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   tm_id                72773 non-null  int64         
 1   club                 72773 non-null  object        
 2   league               72773 non-null  object        
 3   season               72773 non-null  category      
 4   mv                   72773 non-null  float64       
 5   name                 72773 non-null  object        
 6   dob                  72453 non-null  datetime64[ns]
 7   nationality          72773 non-null  object        
 8   continent            72773 non-null  category      
 9   field_position       72773 non-null  category      
 10  main_field_position  72773 non-null  category      
 11  age                  72453 non-null  float64       
 12  year                 72773 non-null  int64         
 13  league_class         72773 n

In [10]:
markval = markval[markval.mv.notnull()]
fig = px.histogram(markval, x= "mv", labels={"mv":"market value"}, 
                   title = "Market value histogram")
fig.show()

As we can see the distribution of the players' market value is highly right skewed as the majority of the players are from non famous leagues and have low market values

In [0]:
markval_plot = markval[markval.mv.notnull()]

In [0]:
markval_plot = markval_plot[markval_plot.age.notnull()]

In [13]:
fig = px.scatter(markval_plot, "age", "mv",hover_data = ["name","age","nationality","club","year"],
                labels = {"mv": "market value", "age" : "age"},
                title = "Market Value vs Age", color = "main_field_position" )
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
corr, _ = pearsonr(markval_plot.age,markval_plot.mv)
print(corr)
fig.show()

Output hidden; open in https://colab.research.google.com to view.

As we can see the correlation between the age and market value is negative. Starting from some moment the market value of the player decreases with the age increasing. Most of the expensive player's are in the age period of 20-26 as in these years their market value increases alongside with the age, as for this age period the player's don't loose in their physical attributes but gain more experience so their market value increases too. As we can see most of the players' market value is less than 100 millions, and only a few players have higher markekt value. Most of these players are young superstars. For example K.Mbappe who is the most expensive player in our dataset has market value of 200 millions at the age of 20. According to his age he has not reached his peak of perfomance yet, but already has a very high market value. Most of the other expensive players are on their peak level of perfomance or very close to it, as they are in 25-28 age period. Examples are Kevin De Bruyne or Antuan Griezmen. As we can se the market value of the players' drastically drops after they turn 30, except some superstars and legends of the game such as Messi and Ronaldo who being aged 32 and 35 have market values of 140 millions and 75 millions accordingly. There are also other aged players who have pretty decent market value for their age such as Karim Benzema(35 millions, 32 years old) or Angel Di Maria(40 millions, 32 years old), but their market value does not overcome 40 millions.

# Summarized market value of the playes' over time

## Market value over age

In [14]:
markval_by_time(markval, "age", "Player's age", 
                "Market value for players over age(mean as summarizer)", "main_field_position")
#age, year

As we can see the players have lowest market values at the beginning and at the end of their career and most of them reach their peak value at the age of 27-31(based on the player's position. The goalkeepers are the oldest to reach their peak value. Also we can see that the market values of the players startes to increase dramaticaly from the late teens and  early twenties of the player up to his peak.

In [15]:
markval_by_time(markval, "age", "Player's age", 
                "Market value over age(median as summarizer)", "main_field_position", summarizer = "median")

Now let's visualize the same plot for all the players and use median as a summarizer as it is more robust to extreme values. The picture is pretty much the same excpet that the player's reach their peak at an older age and the peak value is 10 times lower compared to the players with outlying market values.

## Market value over years

In [16]:
markval_by_time(markval_out, "year", "Years", 
                "Market value for outlying players over years(mean as summarizer)", "main_field_position", 
                summarizer = "mean")

Now let's take a look at market value's behaviour over years. Almost all the positions started to have rise in market value starting from 2007. Goalkeepers and defenders did not lose track and increased in value most of the time up to 2020. However there is a noticable pattern. As we can see in general almost every two years the player's mv rises compared to previous year. What can be the explanation ?. International tournaments. Most of the major international tournaments take place every 4 years, so in every 2 years there is either a world cup or a continental cup. During the tournaments most of the player's who play good earn high market value increases, and we can see that 2014, when world cup in brasil took place, the midfielders had an increase in their market value compared to 2013 and then a decrease in 2015. The same pattern is repeated for attackers in 2010. However, after 2016 all the players started to have increase in thier market value in general, with only once having a decrease(for goalkeepers in 2018). The goalkeepers were mostly stable in terms of rising their market value, having a notable drop in 2015(again after world cup).![alt text](https://cdn.statcdn.com/Infographic/images/normal/14708.jpeg) <br>
This chart explains the goalkeepers mv behaviour for the years 2014 and 2018. The reason is that the world cups that took place during those years were the ones with the highest number of goals scored on average in our dataset.

In [17]:
markval_by_time(markval, "year", "Years", 
                "Market value for all players over years(mean as summarizer)", "main_field_position", 
                summarizer = "mean")

As we can see if we include all the players from the dataset, the international cups effect is not very often anymore, as most of the player's are playing for non famous leagues and weak countries which play in international or continentnal tournaments very rarely.

In [18]:
markval_by_time(markval, "year", "Years", 
                "Market value over years(median as summarizer)", "main_field_position", summarizer = "median")

As we can see usage of median on the whol dataset's market value, underlinese the international cup's effect on attackers for years 2010, 2014 and 2016. As median is robust to extreme values we can see that efter 2018,2019 the player's started to have drops in their market values, whereas the situation was the opposite when using mean as a summarizer, for the (outlying) players. This means that after 2018 players with extreme market values occur more frequently.

In [19]:
markval_by_time(markval, "year", "Years", 
                "Market value over years(range as summarizer)", "main_field_position", summarizer = "range")

In [20]:
markval_by_time(markval, "year", "Years", 
                "Market value over years(max as summarizer)", "main_field_position", summarizer = "max")

As we can see over time the maximal market value seemed to increase equally and stayed stable for a period of 4 years from 2014 - 2018. The main breakpoints of decrease-increase took place in 2018, and 2010, both world cup years, but 2018 was also specific for having the record breaking transfer in season 2017-2018 when Neymar was signed by PSG for record breaking 222M euros wheras the record fee before this tranfer was Paul Pogba's transfer for 105M euros. After Neymar's trasnfer the market changed dramatically and almost all the player's had increases in their estimated market value.

In [21]:
markval_by_time(markval, "year", "Years", 
                "Market value over years(min as summarizer)", "main_field_position", summarizer = "min")

## Market value over age in continents and leagues

As we can see taking range statitstic as a summarizer shows almost the same output as taking the max, whereas min as a summarizer shows a dramatically different picture. If in case of maximum values the market values startet to increase dramatically after 2018, the minimum's case has the opposite pattern, where the highest value lies in 2008. 

In [22]:
markval_by_time(markval_out, "age", "Player's age", 
                "Market value over age by continents(mean as summarizer)", "continent", facet = True)

If we take mean as a summarizer for market values, we can see that the African players have the highest market value in comparison to players from other continetns. Asian players have the lowest values as the majority of them play in their domestic championships. The players from North America and Oceania do not have high market either, as they also mostly play in their domestic leagues. North American players are the youngest to retire in comparison with other leagues, wheras Asians are the oldest to retire, our dataset contains a Japanese player who is 53 years old. The market value's behaviour is very similar for South American and European players, except the fact that young players from South America are valued more expensive than the European players with the same age.

In [23]:
markval.league.unique()

array(['Premier League', 'Eredivisie', 'Bundesliga', 'Championship',
       'Ekstraklasa', 'Ligue 1', 'Serie A', 'Süper Lig', 'Liga NOS',
       '3. Liga', 'League Two', '2. Bundesliga', 'LaLiga', 'Allsvenskan',
       'Premiership', 'Série A', 'League One', 'Superligaen',
       'Eliteserien', 'Serie B', 'Ligue 2', 'Superliga', 'LaLiga2',
       'Jupiler Pro League', 'J1 League', 'Super League',
       'Professional League', 'Liga MX Clausura', 'A-League',
       'Liga DIMAYOR I', 'K League 1', 'MLS'], dtype=object)

In [0]:
top_leagues = ['LaLiga','Bundesliga','Serie A','Premier League',
          'Ligue 1','Liga NOS','Eredivisie','Liga MX Clausura',
          'MLS','Jupiler Pro League', 'Série A', 'Superliga']
markval_tt = markval[markval.league.isin(top_leagues)]

In [25]:
markval_by_time(markval_tt, "age", "Player's age", 
                "Market value over age(mean as summarizer) in top leagues", "league", facet = True, legend = False)

If we take a look at the market value change over player's age in the most popular leagues, we can see that players in League 1(French league) have the highest market value at the youngest age(mean taken as summarizer). However players older than 18 do not have higher market value than their agemates from other leagues. The young players also have good starting point in Liga NOS and Serie A.(Portuguese and Brazilian). These leagues are famous to the whole world for exporting the youngstars to top european leagues. Old players have no value in any league. We can see that player's have significant increase in their market value only in top 5 soccer leagues. The top 5 league player's market value has almost the same path in terms of increasing and decreasing along age, and the most expensive players are in LaLiga and Premier League. The situation in other leageues does not change almost at all as the mean values of the players reach only about 2-3 millions in best cases.

# Player's market value vs player's previous year's and cumulative market value

In [0]:
# markval_prev = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Eda/markval_prev_eda.pkl")
markval_prev = markval

In [27]:
scatter_mv_change(markval_prev, xvar = "last_year_mv", yvar = "mv", xlabel = "last year mv", ylabel = "market value",
                  facet_var = "year", title = "Player's market value vs his previous year's market value")

Output hidden; open in https://colab.research.google.com to view.

As we can see the player's previous year market value is very highly correlated to his current market value. The correlation seems higher for early years.

In [28]:
scatter_mv_change(markval_prev, xvar = "cum_mv", yvar = "mv", xlabel = "cumulative mv", ylabel = "market value",
                  facet_var = "year", title = "Player's market value vs his cumulative market value")

Output hidden; open in https://colab.research.google.com to view.

As we can see before the middle 2010s the correlation between the player's seasonal market value and the cumulative mean of his market value at the time of the transfer was very high, meaning that seasonal based increases or decreases in the players' market value were rare. However, we can see that after around 2011, the seasonal market values of the players started to have a little deviation from their cumulative market value meaning that over time more seasonal based overvalued or undervalued players occurred

In [30]:
scatter_mv_change(markval_prev, xvar = "last_year_mv", yvar = "mv", xlabel = "last year mv", ylabel = "market value",
                  facet_var = "main_field_position", title = "Player's market value vs his last year's  market value")

Output hidden; open in https://colab.research.google.com to view.

In [32]:
scatter_mv_change(markval_prev, xvar = "cum_mv", yvar = "mv", xlabel = "cumulative mv", ylabel = "market value",
                  facet_var = "main_field_position", title = "Player's market value vs his cumulative  market value")

Output hidden; open in https://colab.research.google.com to view.

According to the visualizations the goalkeepers are the most stable in terms of having a current market value close to his previous year's or to his cumulative market value. The situation is similar for defenders with more players being overvalued in compariosn to previous year. The attackers and midfielders have many overvalued,undervalued players and also have more outliers in comparison to other positions.

In [0]:
markval_corr = markval_prev.groupby('year')[['mv','last_year_mv']].corr()
markval_corr.reset_index(inplace = True, drop = False)
markval_corr = markval_corr[markval_corr.level_1 == "last_year_mv"]
markval_corr.drop(["level_1","last_year_mv"], axis = 1, inplace = True)
markval_corr.columns = ["year", "corr_prev_year"]
markval_corr.reset_index(inplace = True, drop = True)

In [34]:
fig = px.line(markval_corr, x = "year", y = "corr_prev_year" ,
              labels = {"corr_prev_year":"Market value correlation with previous year's value"},
              title = "Correlation of market value with previous year's market value over time")
fig.update_xaxes(nticks=14)
fig.show()

As we can see the correlation between player's market value and his previous year's market value was the highest in eraly years and started to drop drastically starting from 2018. This behaviour is explained by the fact that in the early years it was more easier to value the players and mostly their valuation was done using trusted approaches build over time, but over time more factors influence the market value of the player so the correlation with previous years value gets lower and lower. Again we can see breakouts in some international competiton years and after Neymar's transfer. 

In [0]:
markval_corr = markval_prev.groupby('year')[['mv','cum_mv']].corr()
markval_corr.reset_index(inplace = True, drop = False)
markval_corr = markval_corr[markval_corr.level_1 == "cum_mv"]
markval_corr.drop(["level_1","cum_mv"], axis = 1, inplace = True)
markval_corr.columns = ["year", "cum_mv"]
markval_corr.reset_index(inplace = True, drop = True)

In [36]:
fig = px.line(markval_corr, x = "year", y = "cum_mv" ,
              labels = {"cum_mv":"Market value correlation with cumulative market value"},
              title = "Correlation of market value with cumulative market value over time")
fig.update_xaxes(nticks=14)
fig.show()

As we can see the correlation between the variables is solid and there were only a fewpeaks of decreases and increases, and we can see that the current trend is decreasing in thecorrelation between the variables as currently more and more one season wonderers appear,who  get  attached  high  market  value  to  them,  having  no  significant  market  value  history.The decrease in the rate of correlation emerged in 2017

In [0]:
# markval.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_detailed.pkl")

In [0]:
# markval_prev.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_with_last.pkl")